In [29]:
import urllib2
import MySQLdb
import os

In [30]:
db = MySQLdb.connect(user="root", host="localhost", passwd="root")
cursor = db.cursor()

In [31]:
base_url = "http://macho.nci.org.au/macho_photometry/"
fdir = "/Users/Nick/Documents/AGBstuff/new_work/samples/macho/"

### Functions

In [32]:
def download_save(url, outfile):
    outfile = fdir+outfile
    infile = urllib2.urlopen(url)
    fout = open(outfile, "wb")
    fout.write(infile.read())
    fout.close()

In [33]:
def unpack(fname):
    gzip = fdir+fname+".gz"
    os.system("gunzip %s" % gzip)

In [48]:
def create_macho_raw():
    sql = "DROP TABLE agbtables.macho_raw;"
    cursor.execute(sql)
    
    sql = "CREATE TABLE agbtables.macho_raw ("
    sql += "macho_cnt BIGINT(8) UNSIGNED AUTO_INCREMENT PRIMARY KEY,"
    sql += "field_id INT, tile_id INT, star_sequence_id INT,"
    sql += "obs_date DOUBLE, obs_id INT, side_of_pier VARCHAR(3),"
    sql += "exp_time INT, airmass DOUBLE, rmag DOUBLE, rerr DOUBLE,"
    sql += "bmag DOUBLE, berr DOUBLE"
    sql += ");"
    cursor.execute(sql)

In [49]:
def ingest(fname):
    sql = "LOAD DATA LOCAL INFILE '%s'" % (fdir+fname)
    sql += " INTO TABLE agbtables.macho_raw FIELDS TERMINATED BY ';' ("
    sql += "@dummy, field_id, tile_id, star_sequence_id, obs_date, obs_id,"
    sql += "side_of_pier, exp_time, airmass, rmag, rerr,"
    sql += "@dummy,@dummy,@dummy,@dummy,@dummy,@dummy,@dummy,@dummy,@dummy,@dummy,@dummy,@dummy,@dummy,"
    sql += "bmag, berr);"
    
    sql_file = open(fdir+"load_data.sql", "w")
    sql_file.write(sql)
    sql_file.close()
    os.system("mysql -u root --password=root < %sload_data.sql" % fdir)

### Execution

In [36]:
download_save(base_url+"F_1/F_1.3319.gz", "F_1.3319.gz")

In [50]:
create_macho_raw()

In [40]:
unpack("F_1.3319")

In [45]:
ingest("F_1.3319")